# 크롤링과 데이터베이스 예제
* cine 21 랭킹사이트 크롤링
    - 사이즈 주소 : http://www.cine21.com/rank/person
    - 요청 방식 확인 방법 : 개발자 모드 들어가서(Command + option + i) Network -> content 페이지 확인
    - Request URL : http://www.cine21.com/rank/person/content
    - Request Method : POST
    - Form data
        - section = 'actor'
        - period_start = '2021-12'
        - gender = 'all'
        - page = 3

### 1. 라이브러리 import

In [26]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   # 정규표현식 라이브러리

### 2. mongodb connection

In [27]:
connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

### 3. 크롤링 주소 requests

In [28]:
cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2021-01'
post_data['gender'] = 'all'
post_data['page'] = 1

res = requests.post(cine21_url, data=post_data)

### 4. parsing 과 데이터 추출

In [29]:
soup = BeautifulSoup(res.content, 'html.parser')

In [30]:
actors = soup.select('li.people_li div.name')

for actor in actors:
    print(actor.text)

김윤석(6편)
조인성(2편)
허준호(2편)
김소진(4편)
황정민(5편)
조우진(7편)
정만식(4편)


### 정규표현식 참고: https://www.fun-coding.org/DS&AL3-4.html

In [31]:
test_data = '주지훈(6편)'
re.sub('\(\w*\)', '', test_data)

'주지훈'

In [32]:
actors = soup.select('li.people_li > div.name')

for actor in actors:
    print(re.sub('\(\w*\)', '', actor.text))

김윤석
조인성
허준호
김소진
황정민
조우진
정만식


### 5. 배우 상세정보 추출

#### 참고 : 특수한 정규 표현식
```html
    - Greedy(.*) vs Non-Greedy(.*?)
    - https://regexr.com
      - <li><span class="tit">직업</span>배우</li>
    - . 문자는 줄바꿈 문자인 \n 을 제외한 모든 문자 한 개를 의미함
    - * 는 앞 문자가 0번 또는 그 이상 반복되는 패턴
```

In [33]:
actors = soup.select('li.people_li div.name')

for actor in actors:
    print(actor.select_one('a'))

<a href="/db/person/info/?person_id=57423">김윤석(6편)</a>
<a href="/db/person/info/?person_id=16259">조인성(2편)</a>
<a href="/db/person/info/?person_id=2425">허준호(2편)</a>
<a href="/db/person/info/?person_id=82573">김소진(4편)</a>
<a href="/db/person/info/?person_id=14268">황정민(5편)</a>
<a href="/db/person/info/?person_id=98611">조우진(7편)</a>
<a href="/db/person/info/?person_id=64972">정만식(4편)</a>


In [108]:
actors = soup.select('li.people_li div.name')

# http://www.cine21.com/db/person/info/?person_id=57423 <- 각 배우 (_id=57423)의 정보를 나타내는 링크 

actors_info_list = []

for actor in actors:
    actor_link = 'http://www.cine21.com/' + actor.select_one('a').attrs['href']     # 'a' 태그의 속성값 찾기
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    actor_info_dict = {}
    
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
        actor_item_value = re.sub('<.*?>','', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value

    actors_info_list.append(actor_info_dict)

actors_info_list

[{'직업': '직업배우',
  '생년월일': '생년월일1968-01-21',
  '성별': '성별남',
  '신장/체중': '신장/체중178cm, 68kg',
  '학교': '학교동의대학교 학사',
  '취미': '취미여행, 낚시',
  '소속사': '소속사심엔터테인먼트'},
 {'다른 이름': '다른 이름趙寅成',
  '직업': '직업배우',
  '생년월일': '생년월일1981-07-28',
  '성별': '성별남',
  '신장/체중': '신장/체중186cm, 72kg',
  '학교': '학교전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '취미모자모으기, 영화보기, 농구',
  '특기': '특기농구, 태권도(공인 4단)',
  '소속사': '소속사싸이더스 HQ'},
 {'원어명': '원어명許俊豪',
  '직업': '직업배우',
  '생년월일': '생년월일1964-03-03',
  '성별': '성별남',
  '신장/체중': '신장/체중180cm, 75kg',
  '학교': '학교서울예술대학 연극과',
  '취미': '취미만화책 보기',
  '특기': '특기야구, 농구',
  '소속사': '소속사지티비엔터테인먼트'},
 {'직업': '직업배우', '성별': '성별여'},
 {'직업': '직업배우',
  '생년월일': '생년월일1970-09-01',
  '성별': '성별남',
  '신장/체중': '신장/체중180cm, 75kg',
  '학교': '학교서울예술대학 연극과 졸업',
  '특기': '특기농구, 악기연주',
  '소속사': '소속사예당엔터테인먼트'},
 {'다른 이름': '다른 이름조신제', '직업': '직업배우', '생년월일': '생년월일1979-01-16', '성별': '성별남'},
 {'직업': '직업배우', '생년월일': '생년월일1974-12-11', '성별': '성별남'}]

### 6. 배우 흥행 지수와 출연 영화 추출

In [122]:
actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')
for index, actor in enumerate(actors):
    print("배우 이름:", re.sub('\(\w*\)','',actor.text))
    print("흥행 지수:", int(hits[index].text.replace(',', '')))
    movie_titles = movies[index].select('li a span')
    
    movie_title_list = []
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    print("출연 영화:", movie_title_list)

배우 이름: 김윤석
흥행 지수: 46574
출연 영화: ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']
배우 이름: 조인성
흥행 지수: 40753
출연 영화: ['모가디슈', '클래식']
배우 이름: 허준호
흥행 지수: 34900
출연 영화: ['모가디슈', '국가부도의 날']
배우 이름: 김소진
흥행 지수: 29097
출연 영화: ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']
배우 이름: 황정민
흥행 지수: 24720
출연 영화: ['인질', '와이키키 브라더스', '바람난 가족', '신세계', '내 생애 가장 아름다운 일주일']
배우 이름: 조우진
흥행 지수: 24056
출연 영화: ['발신제한', '도굴', '킹메이커', '서복', '국가부도의 날', '봉오동 전투']
배우 이름: 정만식
흥행 지수: 23269
출연 영화: ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']


### 7. 수집한 데이터 기반 딕셔너리 만들기

In [129]:
actors_info_list = []

actors = soup.select('li.people_li div.name')
hits = soup.select('ul.num_info > li > strong')
movies = soup.select('ul.mov_list')

for index, actor in enumerate(actors):
    
    # 전체 배우의 기본 정보 크롤링
    actor_name = re.sub('\(\w*\)','',actor.text)
    actor_hits = int(hits[index].text.replace(',', ''))
    movie_titles = movies[index].select('li a span')
    
    movie_title_list = []
    
    for movie_title in movie_titles:
        movie_title_list.append(movie_title.text)
    movie_title_list
    
    actor_info_dict = {}
    actor_info_dict["배우이름"] = actor_name
    actor_info_dict["흥행지수"] = actor_hits
    actor_info_dict["출연영화"] = movie_title_list
    
    # 각 배우별 링크로 이동 후 자세한 정보 데이터 크롤링
    actor_link = 'http://www.cine21.com/' + actor.select_one('a').attrs['href']
    response_actor = requests.get(actor_link)
    soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
    default_info = soup_actor.select_one('ul.default_info')
    actor_details = default_info.select('li')
    
    # 각 배우별 자세한 정보 리스트를 각 각 크롤링
    for actor_item in actor_details:
        actor_item_field = actor_item.select_one('span.tit').text
        actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
        actor_item_value = re.sub('<.*?>','', actor_item_value)
        actor_info_dict[actor_item_field] = actor_item_value

    actors_info_list.append(actor_info_dict)

actors_info_list

[{'배우이름': '김윤석',
  '흥행지수': 46574,
  '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'],
  '직업': '배우',
  '생년월일': '1968-01-21',
  '성별': '남',
  '신장/체중': '178cm, 68kg',
  '학교': '동의대학교 학사',
  '취미': '여행, 낚시',
  '소속사': '심엔터테인먼트'},
 {'배우이름': '조인성',
  '흥행지수': 40753,
  '출연영화': ['모가디슈', '클래식'],
  '다른 이름': '趙寅成',
  '직업': '배우',
  '생년월일': '1981-07-28',
  '성별': '남',
  '신장/체중': '186cm, 72kg',
  '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)',
  '취미': '모자모으기, 영화보기, 농구',
  '특기': '농구, 태권도(공인 4단)',
  '소속사': '싸이더스 HQ'},
 {'배우이름': '허준호',
  '흥행지수': 34900,
  '출연영화': ['모가디슈', '국가부도의 날'],
  '원어명': '許俊豪',
  '직업': '배우',
  '생년월일': '1964-03-03',
  '성별': '남',
  '신장/체중': '180cm, 75kg',
  '학교': '서울예술대학 연극과',
  '취미': '만화책 보기',
  '특기': '야구, 농구',
  '소속사': '지티비엔터테인먼트'},
 {'배우이름': '김소진',
  '흥행지수': 29097,
  '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년'],
  '직업': '배우',
  '성별': '여'},
 {'배우이름': '황정민',
  '흥행지수': 24720,
  '출연영화': ['인질', '와이키키 브라더스', '바람난 가족', '신세계', '내 생애 가장 아름다운 일주일'],
  '직업': '배우',
  '생년월일': '1970-09-0

### 8. 여러 페이지의 배우 상세  추출하기

In [154]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

actors_info_list = []

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2021-01'
post_data['gender'] = 'all'

for index in range(1, 21):

    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('ul.people_list > li.people_li > span.grade')
    
    for index, actor in enumerate(actors):

        # 전체 배우의 기본 정보 크롤링
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')

        movie_title_list = []

        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        movie_title_list

        actor_info_dict = {}
        actor_info_dict["배우이름"] = actor_name
        actor_info_dict["흥행지수"] = actor_hits
        actor_info_dict["출연영화"] = movie_title_list
        actor_info_dict["랭킹"] = rankings[index].text

        # 각 배우별 링크로 이동 후 자세한 정보 데이터 크롤링
        actor_link = 'http://www.cine21.com/' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        # 각 배우별 자세한 정보 리스트를 각 각 크롤링
        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
            actor_item_value = re.sub('<.*?>','', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value

        actors_info_list.append(actor_info_dict)

print(actors_info_list)

[{'배우이름': '김윤석', '흥행지수': 46574, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}, {'배우이름': '조인성', '흥행지수': 40753, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '다른 이름': '趙寅成', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ'}, {'배우이름': '허준호', '흥행지수': 34900, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}, {'배우이름': '김소진', '흥행지수': 29097, '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년'], '랭킹': '4', '직업': '배우', '성별': '여'}, {'배우이름': '황정민', '흥행지수': 24720, '출연영화': ['인질', '와이키키 브라더스', '바람난 가족', '신세계', '내 생애 가장 아름다운 일주일'], '랭킹': '5', '직업': '배우', '생년월일': '1970-09-01', '성별': '남', '신장/체중': '18

### 9. mongodb에 크롤링 데이터 저장하기

In [155]:
connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

In [156]:
actor_collection.delete_many({})

### 10. 배우 랭킹 및 상세 정보 크롤링 및 mongodb 저장

In [157]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

actors_info_list = []

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2021-01'
post_data['gender'] = 'all'

for index in range(1, 21):

    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('ul.people_list > li.people_li > span.grade')
    
    for index, actor in enumerate(actors):

        # 전체 배우의 기본 정보 크롤링
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')

        movie_title_list = []

        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        movie_title_list

        actor_info_dict = {}
        actor_info_dict["배우이름"] = actor_name
        actor_info_dict["흥행지수"] = actor_hits
        actor_info_dict["출연영화"] = movie_title_list
        actor_info_dict["랭킹"] = rankings[index].text

        # 각 배우별 링크로 이동 후 자세한 정보 데이터 크롤링
        actor_link = 'http://www.cine21.com/' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        # 각 배우별 자세한 정보 리스트를 각 각 크롤링
        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
            actor_item_value = re.sub('<.*?>','', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value

        actors_info_list.append(actor_info_dict)

actor_collection.insert_many(actors_info_list)

#### 이번 강의에서 배운 점

- 정규표현식 활용
- POST 방식 데이터를 웹크롤링 하기
- 파이썬 enumerate 기능 활용
- CSS SELECTOR 활용



In [159]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

actor_collection.find_one({})

In [160]:
docs = actor_collection.find({}).limit(3)

for doc in docs:
    print(doc)

{'_id': ObjectId('61d6a20394d466efcb8f29ca'), '배우이름': '김윤석', '흥행지수': 46574, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}
{'_id': ObjectId('61d6a20394d466efcb8f29cb'), '배우이름': '조인성', '흥행지수': 40753, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '다른 이름': '趙寅成', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ'}
{'_id': ObjectId('61d6a20394d466efcb8f29cc'), '배우이름': '허준호', '흥행지수': 34900, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}
